In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
import urllib.request

In [2]:
location = '/home/simon/Documents/Bodies/data/UCDP'

In [3]:
def get_data(location):

    # Getting and loading views data
    print('Beginning file download UCDP...')

    url_ucdp = 'https://ucdp.uu.se/downloads/ged/ged201-csv.zip'
    path_ucdp = location + "/ged201-csv.zip"
    urllib.request.urlretrieve(url_ucdp, path_ucdp)


    # Getting and loading prio data
    print('Beginning file download PRIO...')

    url_prio = 'http://file.prio.no/ReplicationData/PRIO-GRID/priogrid_shapefiles.zip'
    path_prio = location + '/priogrid_shapefiles.zip'
    urllib.request.urlretrieve(url_prio, path_prio)

    # And move to correct location on computerome
    # gpd.read_file('zip://' + path_prio)
    prio_grid = gpd.read_file('zip://' + path_prio)
    ucdp = pd.read_csv(path_ucdp)

    return prio_grid, ucdp

In [4]:
prio_grid, ucdp = get_data(location)

Beginning file download UCDP...
Beginning file download PRIO...


/home/simon/anaconda3/envs/geo_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [205]:
ucdp_tmp1 = ucdp.copy()

# debug
# print(np.sort(ucdp_tmp1['year'].unique()))

ucdp_tmp1['year_months_start'] = ucdp_tmp1['date_start'].str.slice(start = 0, stop = 7) # Date YYYY-MM-DD
ucdp_tmp1['year_months_end'] = ucdp_tmp1['date_start'].str.slice(start = 0, stop = 7) # Date YYYY-MM-DD


mask1 = (ucdp_tmp1['year'] != ucdp_tmp1['year_months_start'].str.slice(start = 0, stop = 4).astype(int))
mask2 = (ucdp_tmp1['year'] != ucdp_tmp1['year_months_end'].str.slice(start = 0, stop = 4).astype(int))

# check problem
# ucdp_tmp1.loc[mask1 | mask2]

# correction. Note that end and start year for the four entries that is corrected is the same.
ucdp_tmp1.loc[mask1 | mask2, 'year'] = ucdp_tmp1.loc[mask1 | mask2,'year_months_start'].str.slice(start = 0, stop = 4).astype(int)

# see that it have been solved
# ucdp_tmp1.loc[mask1 | mask2]

# debug
# print(np.sort(ucdp_tmp1['year'].unique()))

In [206]:
# but do you need all years or just those in theb Bodies data?
# tjeck woth views month id.. could be nice to just nail that now.

ds_uniques = ucdp_tmp1['date_start'].str.slice(start = 0, stop = 7).unique()
de_uniques = ucdp_tmp1['date_end'].str.slice(start = 0, stop = 7).unique() # do you need both?

months_unique = np.union1d(ds_uniques, de_uniques)
months_unique.sort()

month_id = np.arange(109, months_unique.shape[0] + 109, 1) # this makes sure the month_id matches that of ViWES replication data. Just in case.

month_df = pd.DataFrame({'month_id' : month_id, 'year_months_start' : months_unique, 'year_months_end' : months_unique})

# I checked. There is no instance where the month id will differ if we take start or end.
ucdp_tmp2 = ucdp_tmp1.merge(month_df[['month_id', 'year_months_start']], how = 'outer', on= 'year_months_start')
#ucdp_tmp3 = ucdp_tmp2.merge(month_df[['month_id', 'year_months_end']], how = 'outer', on= 'year_months_end')
#ucdp_tmp3[ucdp_tmp3['month_id_x'] == ucdp_tmp3['month_id_y']] # None


In [192]:
## just a test:

# External
# pkl_file = open('/media/simon/Seagate Backup Plus Drive/Articles/conflict_prediction/data/computerome/currents/preds_df_tt.pkl', 'rb')
# currents_df = pickle.load(pkl_file)
# pkl_file.close()

# for y in np.sort(ucdp_tmp2['year'].unique())[1:29]:

#     check = np.all(np.sort(ucdp_tmp2[ucdp_tmp2['year']==y]['month_id'].unique()) == currents_df[currents_df['year'] == y]['X'].unique())

#     if check == False:

#         print(f"{y}:{np.sort(ucdp_tmp2[ucdp_tmp2['year']==y]['month_id'].unique())}")
#         print(f"{y}:{currents_df[currents_df['year'] == y]['X'].unique()}")
#         print('\n')

# it is cool.

In [207]:
for y in np.sort(ucdp_tmp2['year'].unique()):

    print(f"{y}:{np.sort(ucdp_tmp2[ucdp_tmp2['year']==y]['month_id'].unique())}")

1989:[109 110 111 112 113 114 115 116 117 118 119 120]
1990:[121 122 123 124 125 126 127 128 129 130 131 132]
1991:[133 134 135 136 137 138 139 140 141 142 143 144]
1992:[145 146 147 148 149 150 151 152 153 154 155 156]
1993:[157 158 159 160 161 162 163 164 165 166 167 168]
1994:[169 170 171 172 173 174 175 176 177 178 179 180]
1995:[181 182 183 184 185 186 187 188 189 190 191 192]
1996:[193 194 195 196 197 198 199 200 201 202 203 204]
1997:[205 206 207 208 209 210 211 212 213 214 215 216]
1998:[217 218 219 220 221 222 223 224 225 226 227 228]
1999:[229 230 231 232 233 234 235 236 237 238 239 240]
2000:[241 242 243 244 245 246 247 248 249 250 251 252]
2001:[253 254 255 256 257 258 259 260 261 262 263 264]
2002:[265 266 267 268 269 270 271 272 273 274 275 276]
2003:[277 278 279 280 281 282 283 284 285 286 287 288]
2004:[289 290 291 292 293 294 295 296 297 298 299 300]
2005:[301 302 303 304 305 306 307 308 309 310 311 312]
2006:[313 314 315 316 317 318 319 320 321 322 323 324]
2007:[325 

In [208]:
ucdp_tmp2[ucdp_tmp2['month_id'] == 131]['year'].unique()

array([1990])

In [ ]:
def trim_ucdp(ucdp):

    ucdp_slim = ucdp[['year','priogrid_gid','best','low','high']]
    ucdp_gid = ucdp_slim.groupby(by=['priogrid_gid','year']).sum().reset_index()
    ucdp_gid.rename(columns={'priogrid_gid':'gid'}, inplace=True)

    ucdp_gid['log_best'] = np.log(ucdp_gid['best'] +1)
    ucdp_gid['log_low'] = np.log(ucdp_gid['low'] +1)
    ucdp_gid['log_high'] = np.log(ucdp_gid['high'] +1)

    return ucdp_gid

In [ ]:

def elong_df(df, df_w_years):

    years = sorted(df_w_years['year'].unique())
    df['year'] = years[0]
    concat_df = df.copy()

    for i,j in enumerate(years[1:]):

        df_temp = df.copy()
        df_temp['year'] = j

        concat_df = pd.concat([concat_df,df_temp])
    
    concat_df.reset_index(inplace = True)
    return concat_df 


In [ ]:

def make_df(prio_grid, ucdp):

    print('Creating DF...')

    ucdp_gid = trim_ucdp(ucdp=ucdp)
    prio_grid_yearly = elong_df(prio_grid, ucdp_gid)

    grid_ucdp =  pd.merge(prio_grid_yearly, ucdp_gid, how = 'left', on = ['gid', 'year'])
    grid_ucdp.fillna({'best' : 0, 'low' : 0, 'high' : 0, 'log_best' : 0, 'log_low' : 0, 'log_high' : 0}, inplace = True)

    grid_ucdp = grid_ucdp[['gid', 'xcoord', 'ycoord', 'year', 'best', 'low', 'high', 'log_best', 'log_low', 'log_high']].copy() # remove the everything also the geo col.

    grid_ucdpS = grid_ucdp.sort_values(['year', 'ycoord', 'xcoord'], ascending = [True, False, True])

    # try to keep the jazz
    #grid_ucdpS = grid_ucdpS[['gid','best', 'low',  'high', 'log_best', 'log_low', 'log_high']].copy() # remove the everything also the geo col. But keep gid. Why not.

    x_dim = grid_ucdp['xcoord'].unique().shape[0]
    y_dim = grid_ucdp['ycoord'].unique().shape[0]
    z_dim = grid_ucdp['year'].unique().shape[0]

    ucpd_vol = np.array(grid_ucdpS).reshape((z_dim, y_dim, x_dim, -1))

    return ucpd_vol